In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pydicom as dicom
import pandas as pd
import numpy as np
from random import randint

import os, sys, time, shutil, scipy, cv2, json, datetime
import numpy as np
import pandas as pd
import PIL.Image
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from tqdm.notebook import tqdm

# ignoring warnings
import warnings
warnings.simplefilter("ignore")

import torch
from torch.utils.data import TensorDataset, DataLoader,Dataset
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data.sampler import SubsetRandomSampler, RandomSampler, SequentialSampler
from torch.optim.lr_scheduler import CosineAnnealingLR
from warmup_scheduler import GradualWarmupScheduler 
import albumentations as A
import geffnet

from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
#Import from keras_preprocessing not from keras.preprocessing
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, r2_score, roc_auc_score

import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers, optimizers, metrics

# Build a baseline fully connected model
from keras import models
from keras import layers

In [3]:
train = pd.read_csv('../data/train.csv', index_col=0)
train.head()

,file,target
0,ISIC_1569119,1
1,ISIC_7536704,1
2,ISIC_3696488,1
3,ISIC_3343475,1
4,ISIC_7785723,1


In [4]:
val = pd.read_csv('../data/val.csv', index_col=0)
val.head()

,file,target
0,ISIC_6767569,1
1,ISIC_7312977,1
2,ISIC_1785627,1
3,ISIC_8066110,1
4,ISIC_8838753,1


In [5]:
test = pd.read_csv('../data/test_df.csv', index_col=0)
test.head()

,file,age,site,lesion_id,gender,target,jpg
0,ISIC_0000000,55.0,anterior torso,NaN,female,0,ISIC_0000000.jpg
1,ISIC_0000001,30.0,anterior torso,NaN,female,0,ISIC_0000001.jpg
2,ISIC_0000002,60.0,upper extremity,NaN,female,1,ISIC_0000002.jpg
3,ISIC_0000003,30.0,upper extremity,NaN,male,0,ISIC_0000003.jpg
4,ISIC_0000004,80.0,posterior torso,NaN,male,1,ISIC_0000004.jpg


In [38]:
WORK_DIR = "../data"
label_col = "target"
img_col = "jpg"
train_folder = "../images/train"
test_folder = "../images/test"

os.listdir(WORK_DIR)

['ISIC_2019_Training_GroundTruth.csv',
 'ISIC_2019_Training_Metadata.csv',
 'ISIC_2020_Test_Metadata.csv',
 'ISIC_2020_Training_Duplicates.csv',
 'ISIC_2020_Training_GroundTruth_v2.csv',
 'test.csv',
 'test_df.csv',
 'train.csv',
 'train_df.csv',
 'val.csv']

In [39]:
train_labels = pd.read_csv(os.path.join(WORK_DIR, "train_df.csv"), index_col=0)
label_names = train_labels[label_col].value_counts().index
label_map = {name:i for (i,name) in enumerate(label_names)}
inv_label_map = {v: k for k, v in label_map.items()}

train_labels['label_name'] = train_labels[label_col].copy()
train_labels[label_col] = train_labels[label_col].map(label_map)
display(train_labels.head())

,file,patient_id,lesion_id,gender,age,site,diagnosis,ben_mal,target,jpg,label_name
0,ISIC_2637011,IP_7279968,IL_7972535,male,45.0,head/neck,unknown,benign,0,ISIC_2637011.jpg,0
1,ISIC_0015719,IP_3075186,IL_4649854,female,45.0,upper extremity,unknown,benign,0,ISIC_0015719.jpg,0
2,ISIC_0052212,IP_2842074,IL_9087444,female,50.0,lower extremity,nevus,benign,0,ISIC_0052212.jpg,0
3,ISIC_0068279,IP_6890425,IL_4255399,female,45.0,head/neck,unknown,benign,0,ISIC_0068279.jpg,0
4,ISIC_0074268,IP_8723313,IL_6898037,female,55.0,upper extremity,unknown,benign,0,ISIC_0074268.jpg,0


In [40]:
# Main parameters
BATCH_SIZE = 8
STEPS_PER_EPOCH = len(train_labels)*0.8 / BATCH_SIZE
VALIDATION_STEPS = len(train_labels)*0.2 / BATCH_SIZE
EPOCHS = 4
TARGET_SIZE = 512

In [41]:
train.target = train.target.astype(str)
test.target = test.target.astype(str)

In [42]:
test.head()

,file,age,site,lesion_id,gender,target,jpg
0,ISIC_0000000,55.0,anterior torso,NaN,female,0,ISIC_0000000.jpg
1,ISIC_0000001,30.0,anterior torso,NaN,female,0,ISIC_0000001.jpg
2,ISIC_0000002,60.0,upper extremity,NaN,female,1,ISIC_0000002.jpg
3,ISIC_0000003,30.0,upper extremity,NaN,male,0,ISIC_0000003.jpg
4,ISIC_0000004,80.0,posterior torso,NaN,male,1,ISIC_0000004.jpg


# Data Augmentation

In [7]:
train_datagen = ImageDataGenerator(rescale = 1./255, 
                                   rotation_range = 60, 
                                   width_shift_range = 0.4, 
                                   height_shift_range = 0.4, 
                                   channel_shift_range = 0.4,
                                   shear_range = 0.4, 
                                   zoom_range = 0.4, 
                                   horizontal_flip = True,
                                   vertical_flip = True)

In [8]:
test_folder = '../split/test'
val_folder = '../split/val'
train_folder = '../split/train'

In [9]:
# get all the data in the directory split/validation (200 images), and reshape them
val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        val_folder, 
        target_size=(64, 64),
        batch_size = 32,
        class_mode='binary')

# get all the data in the directory split/train (542 images), and reshape them
train_generator = train_datagen.flow_from_directory(
        train_folder, 
        target_size=(64, 64), 
        batch_size = 32, 
        class_mode='binary')

Found 6626 images belonging to 2 classes.
Found 26500 images belonging to 2 classes.


In [13]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64 ,64,  3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(32, (4, 4), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

opt = optimizers.Adam(learning_rate=0.02)

model.compile(loss='binary_crossentropy',
              optimizer= 'adam',
              metrics=['accuracy', tf.metrics.AUC(), tf.metrics.TruePositives(), tf.metrics.TrueNegatives()])

In [18]:
history_2 = model.fit_generator(train_generator,
                                steps_per_epoch=20,
                                epochs=30, 
                                validation_data=val_generator, 
                                validation_steps=25)

Epoch 1/30
 6/20 [========>.....................] - ETA: 2:40 - loss: 0.1069 - accuracy: 0.9792 - auc: 0.5106 - true_positives: 188.0000 - true_negatives: 0.0000e+00

UnknownError:  OSError: image file is truncated (11 bytes not processed)
Traceback (most recent call last):

  File "/Users/examsherpa/opt/anaconda3/envs/nn-env/lib/python3.6/site-packages/tensorflow/python/ops/script_ops.py", line 249, in __call__
    ret = func(*args)

  File "/Users/examsherpa/opt/anaconda3/envs/nn-env/lib/python3.6/site-packages/tensorflow/python/autograph/impl/api.py", line 620, in wrapper
    return func(*args, **kwargs)

  File "/Users/examsherpa/opt/anaconda3/envs/nn-env/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 891, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/Users/examsherpa/opt/anaconda3/envs/nn-env/lib/python3.6/site-packages/tensorflow/python/keras/engine/data_adapter.py", line 807, in wrapped_generator
    for data in generator_fn():

  File "/Users/examsherpa/opt/anaconda3/envs/nn-env/lib/python3.6/site-packages/tensorflow/python/keras/engine/data_adapter.py", line 933, in

In [ ]:
test_x, test_y = next(test_generator)

In [ ]:
results_test = model.evaluate(test_x, test_y)

In [ ]:
results_test

In [ ]:
# Generate test set for data visualization
test_set = testing_data_generator. \
               flow_from_directory('Dataset/PetImages/Test/',
                                    target_size = (INPUT_SIZE,INPUT_SIZE),
                                    batch_size = 1,
                                    class_mode = 'binary')

strongly_wrong_idx = []
strongly_right_idx = []
weakly_wrong_idx = []

for i in range(test_set.__len__()):
    img = test_set.__getitem__(i)[0]
     img = test_set.__getitem__(i)[0]
    pred_prob = model.predict(img)[0][0]
    pred_label = int(pred_prob > 0.5)
    actual_label = int(test_set.__getitem__(i)[1][0])
    if pred_label != actual_label and (pred_prob > 0.8 or 
        pred_prob < 0.2): strongly_wrong_idx.append(i)
    elif pred_label != actual_label and (pred_prob > 0.4 and 
        pred_prob < 0.6): weakly_wrong_idx.append(i)
    elif pred_label == actual_label and (pred_prob > 0.8 or
        pred_prob < 0.2): strongly_right_idx.append(i)
    # stop once we have enough images to plot
    if (len(strongly_wrong_idx)>=9 and len(strongly_right_idx)>=9 
        and len(weakly_wrong_idx)>=9): break


from matplotlib import pyplot as plt
import random

def plot_on_grid(test_set, idx_to_plot, img_size=INPUT_SIZE):
    fig, ax = plt.subplots(3,3, figsize=(20,10))
    for i, idx in enumerate(random.sample(idx_to_plot,9)):
        img = test_set.__getitem__(idx)[0].reshape(img_size, img_size ,3)
        ax[int(i/3), i%3].imshow(img)
        ax[int(i/3), i%3].axis('off')



In [63]:
import glob, pylab, pandas as pd
import pydicom, numpy as np
from os import listdir
from os.path import isfile, join
import matplotlib.pylab as plt
import matplotlib.pyplot as plt2
from plotly.offline import init_notebook_mode
import plotly.graph_objs as go
from plotly import tools
import os
import seaborn as sns
from keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from keras.applications import DenseNet121
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import Adam
import tensorflow as tf
from plotly.tools import FigureFactory as FF
import plotly
from tqdm import tqdm
import cv2
from PIL import Image
from plotly.offline import iplot
import cufflinks
import cv2 as cv
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
%load_ext autoreload
plotly.io.templates.default = "none"
import altair as alt; import altair_render_script
plt.style.use("fivethirtyeight")
def _generate_bar_plot_hor(df, col, title, color, w=None, h=None, lm=0, limit=100):
    cnt_srs = df[col].value_counts()[:limit]
    trace = go.Bar(y=cnt_srs.index[::-1], x=cnt_srs.values[::-1], orientation = 'h',
        marker=dict(color=color))

    layout = dict(title=title, margin=dict(l=lm), width=w, height=h)
    data = [trace]
    annotations = []
    annotations += [go.layout.Annotation(x=673, y=100, xref="x", yref="y", text="(Most Popular)", showarrow=False, arrowhead=7, ax=0, ay=-40)]
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)
    
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

ModuleNotFoundError: No module named 'cufflinks'